In [1]:
from Bio import Entrez
import pandas as pd
from tqdm.notebook import tqdm

Entrez.email = "singhmohit.1889@gmail.com"
Entrez.api_key = "1c22910bfa9f49693b451d3dfec15c1dbb09"

def extract_pubmed_id(reference):
    try:
        for article_id in reference['ArticleIdList']:
            if article_id.attributes['IdType'] == 'pubmed':
                return str(article_id)
    except:
        return float('NaN')

def fetch_paper_details(pmid):
    handle = Entrez.efetch(db="pubmed", id=pmid, rettype="xml", retmode="text")
    records = Entrez.read(handle)
    handle.close()
    
    paper_title = pmid
#     paper_title = records['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle']
    authors = records['PubmedArticle'][0]['MedlineCitation']['Article']['AuthorList']
    paper_authors = [f"{author['ForeName']} {author['LastName']}" for author in authors]
    journal = records['PubmedArticle'][0]['MedlineCitation']['Article']['Journal']['Title']
    year = records['PubmedArticle'][0]['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate']['Year']   
    return paper_title, paper_authors, journal, year

def fetch_citations(pmid):
    handle = Entrez.elink(dbfrom="pubmed", id=pmid, linkname="pubmed_pubmed_citedin")
    records = Entrez.read(handle)
    handle.close()
    
    citing_paper_titles = []
    citing_paper_references = []
    details = []
    if records[0]["LinkSetDb"]:
        for link in records[0]["LinkSetDb"][0]["Link"]:
            citing_pmid = link["Id"]
            handle = Entrez.efetch(db="pubmed", id=citing_pmid, rettype="xml", retmode="text")
            records = Entrez.read(handle)
            handle.close()
            
            paper_title = citing_pmid
#             paper_title = records['PubmedArticle'][0]['MedlineCitation']['Article']['ArticleTitle']
            references = records['PubmedArticle'][0]['PubmedData']['ReferenceList'][0]['Reference']
            reference_list = [extract_pubmed_id(reference) for reference in references]
            details.append((paper_title,reference_list))
    
    return details

In [2]:
df = pd.read_csv("savedrecs imp csv.csv",encoding='latin-1')
PubmedId = df[~df['Pubmed Id'].isna()]['Pubmed Id'].values

In [3]:
len(PubmedId)

97

In [4]:
PubmedId = PubmedId.astype(int)
PubmedId = PubmedId.astype(str)

In [5]:
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Function to fetch details and citations
def fetch_details_and_citations(pmid):
    try:
        paper_title, paper_authors, journal, year = fetch_paper_details(pmid)
        citing_papers = fetch_citations(pmid)
        return (pmid, paper_title, paper_authors, journal, year, citing_papers)
    except Exception as e:
        return (pmid, None, None, None, None, None)

# Initialize DataFrame
df = pd.DataFrame([], columns=['Paper_Title', 'Paper_Authors', 'Journal', 'Year', 'Citing_Paper_Title', 'Citing_Paper_References'])

# List of PubMed IDs
# PubmedId = PubmedId

# Use ThreadPoolExecutor for parallel processing
with ThreadPoolExecutor(max_workers=8) as executor:
    futures = {executor.submit(fetch_details_and_citations, pmid): pmid for pmid in PubmedId}
    
    for future in tqdm(as_completed(futures), total=len(PubmedId)):
        pmid, paper_title, paper_authors, journal, year, citing_papers = future.result()
        if paper_title and paper_authors and journal and year and citing_papers:
            for title, ref in citing_papers:
                df.loc[len(df)] = [paper_title, paper_authors, journal, year, title, ref]
        else:
            print(f"Failed to fetch details for PMID: {pmid}")

# Save the DataFrame to a CSV file or any other desired format
df.to_csv('outputPMID.csv', index=False)


  3%|██▍                                                                             | 3/97 [00:07<03:10,  2.03s/it]

Failed to fetch details for PMID: 34910933


 24%|██████████████████▋                                                            | 23/97 [05:16<15:20, 12.44s/it]

Failed to fetch details for PMID: 32783919


 38%|██████████████████████████████▏                                                | 37/97 [15:05<30:51, 30.87s/it]

...

 70%|███████████████████████████████████████████████████████▍                       | 68/97 [27:08<02:24,  4.97s/it]

Failed to fetch details for PMID: 7546706


 75%|███████████████████████████████████████████████████████████▍                   | 73/97 [27:24<01:42,  4.28s/it]

Failed to fetch details for PMID: 32998157


100%|███████████████████████████████████████████████████████████████████████████████| 97/97 [42:00<00:00, 25.99s/it]


In [17]:
df['Citing_Paper_References'] = df['Citing_Paper_References'].apply(lambda x: [i for i in x if isinstance(i,str)])

In [19]:
d = df.copy()

In [20]:
df = df.drop('Paper_Authors',axis=1)

In [24]:
df.head()

,Paper_Title,Journal,Year,Citing_Paper_Title,Citing_Paper_References
0,37020354,Annals of the New York Academy of Sciences,2023,38125026,"[31869338, 26229114, 27153120, 25121750, 21987..."
1,37020354,Annals of the New York Academy of Sciences,2023,37084166,"[35454874, 35089462, 36675082, 37534923, 27531..."
2,37371829,Biomedicines,2023,38400169,"[35821637, 37705354, 36311701, 37595605, 37454..."
3,37371829,Biomedicines,2023,38254150,"[33049211, 33942880, 27498188, 28762175, 34706..."
4,35536311,Nucleic acids research,2022,38406265,"[8559254, 8674114, 8797828, 9106664, 37492704,..."


In [39]:
df.shape

(7205, 5)

In [40]:
df_exploded = df.explode('Citing_Paper_References').reset_index(drop=True)

In [48]:
df_exploded.columns = ['Focal Paper Pubmed ID','Journal','Year',
                       'Citing Paper Pubmed ID',
                       'Citing paper references Pubmed ID']

In [49]:
df_exploded.to_csv('with_pmid_v1.csv',index=False)

In [50]:
df_exploded

,Focal Paper Pubmed ID,Journal,Year,Citing Paper Pubmed ID,Citing paper references Pubmed ID
0,37020354,Annals of the New York Academy of Sciences,2023,38125026,31869338
1,37020354,Annals of the New York Academy of Sciences,2023,38125026,26229114
2,37020354,Annals of the New York Academy of Sciences,2023,38125026,27153120
3,37020354,Annals of the New York Academy of Sciences,2023,38125026,25121750
4,37020354,Annals of the New York Academy of Sciences,2023,38125026,21987658
...,...,...,...,...,...
730912,16111635,Immunity,2005,16330812,15563593
730913,16111635,Immunity,2005,16330812,15208624
730914,16111635,Immunity,2005,16330812,16027240
730915,16111635,Immunity,2005,16330812,16230478
